In [1]:
import os
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from dataset import EEGDataset
from torch.utils.data import random_split

C:\Users\Aksel\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Aksel\AppData\Local\Programs\Python\Python39\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# load in the dataset

raw_data_dir = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple'

ds1 = EEGDataset(raw_data_dir,4, 250) #Instantiate a dataset using the directory of data, amount of night to include and amount of samples in a segment

//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_1\artefact_annotations.npy
Lables for night 0 loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_1\EEG_raw_250hz.npy
Night 0 data loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_2\artefact_annotations.npy
Lables for night 1 loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_2\EEG_raw_250hz.npy
Night 1 data loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_3\artefact_annotations.npy
Lables for night 2 loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_3\EEG_raw_250hz.npy
Night 2 data loaded
//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple\S_01\night_

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
print(f'Using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.BatchNorm1d(250),
            nn.Linear(250, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cuda device


In [9]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

model = NeuralNetwork()

#Weighting artefacts 10x
#loss = nn.BCELoss(weight = torch.tensor([0.1,0.9]))

#weights = [1,10]
#class_weights = torch.FloatTensor(weights)#.cuda()

#loss = nn.CrossEntropyLoss(weight=class_weights) # Using cross entropy and not binary cross entropy because weight implementation is better :/
#loss = nn.CrossEntropyLoss(class_weights)

#pos_weight: amount of positive examples compared to negative examples. Calculate as: negative_examples/positive_examples
loss = nn.BCEWithLogitsLoss(pos_weight = 10*torch.ones([batch_size])) 

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X).reshape(-1)# Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
        yFloat = y.type(torch.FloatTensor)

        try:
            loss = loss_fn(pred, yFloat)
        except:
            print("loss function failed")
            break


        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            print("Predicted values:")
            print(pred)
            print("Actual values:")
            print(yFloat)



def test_loop(dataloader_test, model, loss_fn, test_set = True):
    size = len(dataloader_test.dataset)
    num_batches = len(dataloader_test)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader_test:
            pred = model(X).reshape(-1) # Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
            test_loss += loss_fn(pred, y.type(torch.FloatTensor)).item()
            correct += (pred.round() == y).type(torch.float).sum().item()
            

    test_loss /= num_batches
    correct /= size
    if test_set:
        print(f"Test set Error: \n Test Set Accuracy: {(100*correct):>0.5f}%, Avg Test Set loss: {test_loss:>8f} \n")
    else:
        print(f"Training Set Error: \n Training Set Accuracy: {(100*correct):>0.5f}%, Avg Training Set loss: {test_loss:>8f} \n")


In [11]:
# Split data into train and test data
trainSamples = int(ds1.__len__()*0.7)
testSamples = int(ds1.__len__() - trainSamples)
training_data, test_data = random_split(ds1, (trainSamples,testSamples), generator=torch.Generator().manual_seed(42))

train_dataloader = DataLoader(training_data, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
test_dataloader = DataLoader(test_data, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss

In [12]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    train_loop(train_dataloader, model, loss, optimizer)    
    test_loop(test_dataloader, model, loss)
    test_loop(train_dataloader, model, loss, test_set = False)

print("Done!")

Epoch 1
-------------------------------
loss: 0.972786  [    0/1906271]
Predicted values:
tensor([0.5174, 0.4962, 0.4829, 0.4889, 0.4847, 0.4962, 0.4944, 0.4833, 0.4778,
        0.4824, 0.4864, 0.5102, 0.4908, 0.4888, 0.4904, 0.4896, 0.5074, 0.4823,
        0.5194, 0.4876, 0.4852, 0.4911, 0.5174, 0.5168, 0.5111, 0.4908, 0.5118,
        0.5055, 0.5181, 0.5170, 0.4962, 0.5211, 0.5132, 0.5086, 0.4876, 0.4811,
        0.4952, 0.4899, 0.4887, 0.5035, 0.4915, 0.4958, 0.4914, 0.4907, 0.5178,
        0.5155, 0.4964, 0.4840, 0.4885, 0.5190, 0.4908, 0.4889, 0.5136, 0.5072,
        0.4789, 0.5104, 0.4815, 0.4818, 0.4962, 0.4962, 0.5082, 0.5100, 0.5169,
        0.4873], grad_fn=<ReshapeAliasBackward0>)
Actual values:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
loss

KeyboardInterrupt: 

In [ ]:
print('debug')